<a href="https://colab.research.google.com/github/rizqul/PukulEnam-recommend-system/blob/main/RecommenderSystem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np
import seaborn as sns

# DATASET NEWSROOM

In [287]:
authors_df = pd.read_csv('/content/drive/MyDrive/Dataset/Newsroom_data.csv')

In [288]:
authors_df.head(10)

,Name,Copywriting,Design,Available Days,Nasional,Internasional,Health,Finance,Technology,Gaming,Sports,Entertainment
0,Aditya,4,2,"Senin, Selasa, Rabu, Kamis, Jumat, Sabtu, Minggu",4,5,5.0,2.0,3.0,0.5,0,0.5
1,Andhika Mifta Alauddin,4,3,"Senin, Selasa, Rabu, Kamis, Jumat, Sabtu, Minggu",5,4,1.0,1.0,3.0,3.0,5,3.0
2,Ni Nyoman Ayu Sintya Dewi,4,3,"Senin, Selasa, Rabu, Jumat, Sabtu",4,3,3.0,4.0,4.0,1.0,2,3.0
3,Dewa Bagus Trima Putra,4,5,"Rabu, Jumat",4,4,3.0,5.0,4.0,3.0,3,5.0
4,Patma Ari Ayu Kartini,4,4,"Selasa, Jumat",4,4,4.0,4.0,4.0,3.0,3,4.0
5,Risa Pebriyanthi,4,2,"Selasa, Sabtu",5,3,3.0,4.5,2.0,1.0,1,2.0
6,Suci Hastika Salma'aini,5,5,"Rabu, Kamis",5,4,3.0,4.5,4.5,2.0,3,3.0
7,Ni Luh Santi Wahyuni,4,2,Rabu,5,3,2.0,4.0,2.0,1.0,1,2.5
8,Andre Winata,4,4,"Senin, Rabu, Minggu",4,4,1.5,2.0,5.0,4.0,3,2.0
9,Iga Narendra Pramawijaya,4,1,"Sabtu, Minggu",3,4,2.0,4.0,5.0,3.0,3,1.0


# Modeling Content-Based Filtering

In [12]:
!pip install Sastrawi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 5.5 MB/s eta 0:00:00


In [13]:
# import library
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import matplotlib.pyplot as plt

import string
import nltk
import re
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

## Predict Topics

In [30]:
def clean_text(text):
  # Remove puncuation
  text = text.translate(string.punctuation)
  # Convert words to lower case and split them
  text = text.lower().split()
  # Remove stop words
  stops = set(stopwords.words("indonesian"))
  text = [w for w in text if not w in stops and len(w) >= 3]
  text = " ".join(text)
  # Clean the text
  text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
  # Stemming
  text = text.split()
  factory = StemmerFactory()
  stemmer = factory.create_stemmer()
  stemmed_words = [stemmer.stem(word) for word in text]
  text = " ".join(stemmed_words)
  return text

def fit_tokenizer(train_sentences, num_words, oov_token):
    tokenizer = Tokenizer(num_words=num_words , oov_token=oov_token)
    tokenizer.fit_on_texts(train_sentences)
    return tokenizer

def seq_and_pad(sentences, tokenizer, padding, maxlen):
    sequences = tokenizer.texts_to_sequences(sentences)
    padded_sequences = pad_sequences(sequences, maxlen=maxlen, padding=padding)
    return padded_sequences

In [486]:
NUM_WORDS = 1000
EMBEDDING_DIM = 100
MAXLEN = 200
PADDING = 'post'
OOV_TOKEN = "<OOV>"
features = ["Finance", "Gaming", "Entertainment", "Health", "Sports", "Technology"]
model = tf.keras.models.load_model('/content/drive/MyDrive/Dataset/predict_topics.h5')

# # Function to predict topic based on news title
# def predict_topic(title):
#   clean = clean_text(title)
#   tokenizer = fit_tokenizer(clean, NUM_WORDS, OOV_TOKEN)
#   padded_sequence = seq_and_pad([clean], tokenizer, PADDING, MAXLEN)
#   prediction = model.predict(padded_sequence)
#   topic_index = np.argmax(prediction)
#   topic = features[topic_index]
#   return topic

In [472]:
def get_topic_preferences(title):
    clean = clean_text(title)
    tokenizer = fit_tokenizer(clean, NUM_WORDS, OOV_TOKEN)
    padded_sequence = seq_and_pad([clean], tokenizer, PADDING, MAXLEN)
    predictions = model.predict(padded_sequence)[0]
    category_probabilities = dict(zip(features, predictions))
    return category_probabilities

In [527]:
news_text = "Pekan Ini, IHSG Ditutup Menurun Hampir 2 Persen"

topic_preferences = get_topic_preferences(news_text)
topic_preferences

1/1 [==============================] - 0s 22ms/step


{'Finance': 0.20540199,
 'Gaming': 0.060506858,
 'Entertainment': 0.042592645,
 'Health': 0.29678062,
 'Sports': 0.14964136,
 'Technology': 0.24507655}

## User Preference

In [507]:
features_cbf = ['Copywriting','Design','Nasional','Internasional','Health','Finance','Technology','Gaming','Sports','Entertainment']

In [508]:
# User Input
user_inputs = ['Copywriting','Internasional']

In [509]:
user_profile = np.array([[1 if feature in user_inputs else (topic_preferences[feature] if feature in topic_preferences else 0) for feature in features_cbf]])
user_profile

array([[1.        , 0.        , 0.        , 1.        , 0.14189866,
        0.16211006, 0.31966969, 0.07280008, 0.24649884, 0.05702265]])

In [510]:
from sklearn.preprocessing import MinMaxScaler
# Normalize feature vectors
scaler = MinMaxScaler()
author_features = scaler.fit_transform(authors_df[features_cbf])

In [511]:
# # 'Copywriting', 'Design', 'Nasional', 'Internasional', 'Health', 'Finance', 'Technology', 'Gaming', 'Sports', 'Entertainment'
# user_preferences = {
#     'Copywriting': user_profile[0],
#     'Design': user_profile[1],
#     # 'Available Days': 'Senin',
#     'Nasional': user_profile[2],
#     'Internasional': user_profile[3],
#     'Health': topic_preferences['Kesehatan'],
#     'Finance': topic_preferences['Finansial'],
#     'Technology': topic_preferences['Teknologi'],
#     'Gaming': topic_preferences['Gaming'],
#     'Sports': topic_preferences['Olahraga'],
#     'Entertainment': topic_preferences['Hiburan']
# }

In [512]:
# user_df = pd.DataFrame(user_preferences, index=[0])
# user_df

## Calculate Similarity

In [513]:
# Calculate cosine similarity between user preferences and items
similarities = cosine_similarity(user_profile, author_features)
similarities

array([[0.74686971, 0.56112646, 0.47443902, 0.52708824, 0.56654878,
        0.4278507 , 0.60488207, 0.42966672, 0.58037512, 0.67235795,
        0.23197064, 0.63390127, 0.57423751, 0.51892272]])

In [514]:
# Sort items based on similarity scores
recommended_items = np.argsort(similarities)[0][::-1]
recommended_items

array([ 0,  9, 11,  6,  8, 12,  4,  1,  3, 13,  2,  7,  5, 10])

In [515]:
# Print recommended
print("Recommended:")
for item in recommended_items:
    print(f"{item}: {authors_df['Name'][item]}")

Recommended:
0: Aditya
9: Iga Narendra Pramawijaya
11: Abiyyu Didar Haq
6: Suci Hastika Salma'aini
8: Andre Winata
12: Putu Gede Arya Karna Sampalan
4: Patma Ari Ayu Kartini
1: Andhika Mifta Alauddin
3: Dewa Bagus Trima Putra
13: Visakha Vidyadevi Wiguna
2: Ni Nyoman Ayu Sintya Dewi
7: Ni Luh Santi Wahyuni 
5: Risa Pebriyanthi
10: Nyoman Satiya Nanjaya Sadha


In [516]:
new_df = authors_df.reindex(list(recommended_items))
new_df

,Name,Copywriting,Design,Available Days,Nasional,Internasional,Health,Finance,Technology,Gaming,Sports,Entertainment
0,Aditya,4,2,"Senin, Selasa, Rabu, Kamis, Jumat, Sabtu, Minggu",4,5,5.0,2.0,3.0,0.5,0,0.5
9,Iga Narendra Pramawijaya,4,1,"Sabtu, Minggu",3,4,2.0,4.0,5.0,3.0,3,1.0
11,Abiyyu Didar Haq,5,5,"Sabtu, Minggu",2,4,5.0,0.0,3.0,3.0,1,0.0
6,Suci Hastika Salma'aini,5,5,"Rabu, Kamis",5,4,3.0,4.5,4.5,2.0,3,3.0
8,Andre Winata,4,4,"Senin, Rabu, Minggu",4,4,1.5,2.0,5.0,4.0,3,2.0
12,Putu Gede Arya Karna Sampalan,4,4,"Jumat, Sabtu",4,4,3.0,5.0,4.0,3.0,4,3.0
4,Patma Ari Ayu Kartini,4,4,"Selasa, Jumat",4,4,4.0,4.0,4.0,3.0,3,4.0
1,Andhika Mifta Alauddin,4,3,"Senin, Selasa, Rabu, Kamis, Jumat, Sabtu, Minggu",5,4,1.0,1.0,3.0,3.0,5,3.0
3,Dewa Bagus Trima Putra,4,5,"Rabu, Jumat",4,4,3.0,5.0,4.0,3.0,3,5.0
13,Visakha Vidyadevi Wiguna,5,5,"Sabtu, Minggu",5,4,5.0,0.0,0.0,0.0,2,5.0


In [517]:
# Filter Writers by Available Days
day = "Sabtu"
filtered_df = new_df[new_df['Available Days'].str.contains(day)]
filtered_df

,Name,Copywriting,Design,Available Days,Nasional,Internasional,Health,Finance,Technology,Gaming,Sports,Entertainment
0,Aditya,4,2,"Senin, Selasa, Rabu, Kamis, Jumat, Sabtu, Minggu",4,5,5.0,2.0,3.0,0.5,0,0.5
9,Iga Narendra Pramawijaya,4,1,"Sabtu, Minggu",3,4,2.0,4.0,5.0,3.0,3,1.0
11,Abiyyu Didar Haq,5,5,"Sabtu, Minggu",2,4,5.0,0.0,3.0,3.0,1,0.0
12,Putu Gede Arya Karna Sampalan,4,4,"Jumat, Sabtu",4,4,3.0,5.0,4.0,3.0,4,3.0
1,Andhika Mifta Alauddin,4,3,"Senin, Selasa, Rabu, Kamis, Jumat, Sabtu, Minggu",5,4,1.0,1.0,3.0,3.0,5,3.0
13,Visakha Vidyadevi Wiguna,5,5,"Sabtu, Minggu",5,4,5.0,0.0,0.0,0.0,2,5.0
2,Ni Nyoman Ayu Sintya Dewi,4,3,"Senin, Selasa, Rabu, Jumat, Sabtu",4,3,3.0,4.0,4.0,1.0,2,3.0
5,Risa Pebriyanthi,4,2,"Selasa, Sabtu",5,3,3.0,4.5,2.0,1.0,1,2.0
10,Nyoman Satiya Nanjaya Sadha,2,2,"Sabtu, Minggu",2,3,4.0,0.0,3.0,3.0,2,0.0


## Recommend Writers

In [518]:
# def writers_recommendation(user_preferences, writers_data, top_n=3):
#     filtered_writers = filter_writers(user_preferences, writers_data)
#     similarity_scores = calculate_similarity(user_preferences, writers_data)

#     # Sort writers based on similarity scores
#     sorted_writers = sorted(zip(filtered_writers, similarity_scores), key=lambda x: x[1], reverse=True)

#     # Get top N recommended writers
#     top_writers = sorted_writers[:top_n]

#     return top_writers

In [519]:
# writers_recommendation(user_df, authors_df, 3)